In [8]:
import numpy as np
import pandas as pd
import os
import glob

In [9]:
BASE_DIR = '..'
RAW_DIR = os.path.join(BASE_DIR, 'raw', 'berlin')
PROCESSED_DIR = os.path.join(BASE_DIR, 'processed', 'berlin')
REPORTS_DIR = os.path.join(BASE_DIR, 'reports')

os.makedirs(PROCESSED_DIR, exist_ok = True)
os.makedirs(REPORTS_DIR, exist_ok = True)

print(f"Sẽ đọc dữ liệu từ: {RAW_DIR}")
print(f"Sẽ lưu kết quả vào: {PROCESSED_DIR} và {REPORTS_DIR}")

Sẽ đọc dữ liệu từ: ..\raw\berlin
Sẽ lưu kết quả vào: ..\processed\berlin và ..\reports


In [ ]:
snapshot_folders = [f for f in glob.glob(os.path.join(RAW_DIR, '*')) if os.path.isdir(f)]  
print(f"Tìm thấy {len(snapshot_folders)} snapshots. Bắt đầu khảo sát TOÀN BỘ 4 TỆP:")

for folder_path in snapshot_folders:
    snapshot_name = os.path.basename(folder_path)
    print(f"\n=============================================")
    print(f"KHẢO SÁT SNAPSHOT: {snapshot_name}")
    print(f"=============================================")
    
    # --- 1. Tệp: listings.csv.gz ---
    print("\n--- 1. Tệp: listings.csv.gz ---")
    listings_file = os.path.join(folder_path, 'listings.csv.gz')
    if os.path.exists(listings_file):
        df_list = pd.read_csv(listings_file)
        df_list.info(verbose=False)
        print(f"   > Kiểu dữ liệu 'price' (listings): {df_list['price'].dtype}")
        print(f"   > Kiểu dữ liệu 'host_since': {df_list['host_since'].dtype}")
    else:
        print("   > LỖI: Không tìm thấy tệp.")
        
    # --- 2. Tệp: calendar.csv.gz ---
    print("\n--- 2. Tệp: calendar.csv.gz ---")
    calendar_file = os.path.join(folder_path, 'calendar.csv.gz')
    if os.path.exists(calendar_file):
        df_cal = pd.read_csv(calendar_file)
        df_cal.info(verbose=False)
        print(f"   > Kiểu dữ liệu 'price' (calendar): {df_cal['price'].dtype}")
        print(f"   > Kiểu dữ liệu 'date': {df_cal['date'].dtype}")
    else:
        print("   > LỖI: Không tìm thấy tệp.")

    # --- 3. Tệp: reviews.csv.gz ---
    print("\n--- 3. Tệp: reviews.csv.gz ---")
    reviews_file = os.path.join(folder_path, 'reviews.csv.gz')
    if os.path.exists(reviews_file):
        df_rev = pd.read_csv(reviews_file)
        df_rev.info() 
    else:
        print("   > LỖI: Không tìm thấy tệp.")
        
    # --- 4. Tệp: neighbourhoods.csv ---
    print("\n--- 4. Tệp: neighbourhoods.csv ---")
    neigh_file = os.path.join(folder_path, 'neighbourhoods.csv')
    if os.path.exists(neigh_file):
        df_neigh = pd.read_csv(neigh_file)
        df_neigh.info()
        print("   > 5 dòng đầu:")
        print(df_neigh.head())
    else:
        print("   > LỖI: Không tìm thấy tệp.")

print("\n=============================================")
print("HOÀN TẤT KHẢO SÁT (Toàn bộ 4 tệp)")
print("=============================================")

Tìm thấy 4 snapshots. Bắt đầu khảo sát TOÀN BỘ 4 TỆP:

KHẢO SÁT SNAPSHOT: 15 March, 2025

--- 1. Tệp: listings.csv.gz ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13945 entries, 0 to 13944
Columns: 79 entries, id to reviews_per_month
dtypes: float64(23), int64(20), object(36)
memory usage: 8.4+ MB
   > Kiểu dữ liệu 'price' (listings): object
   > Kiểu dữ liệu 'host_since': object

--- 2. Tệp: calendar.csv.gz ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5085865 entries, 0 to 5085864
Columns: 7 entries, listing_id to maximum_nights
dtypes: float64(3), int64(1), object(3)
memory usage: 271.6+ MB
   > Kiểu dữ liệu 'price' (calendar): object
   > Kiểu dữ liệu 'date': object

--- 3. Tệp: reviews.csv.gz ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588592 entries, 0 to 588591
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     588592 non-null  int64 
 1   id             588592 non

In [12]:
def clean_price(price_value):
    if isinstance(price_value, (int, float)):
        if pd.isna(price_value):
            return np.nan
        return float(price_value)
    
    if isinstance(price_value, str):
        cleaned_str = price_value.replace('$', '').replace(',', '')
        try:
            return float(cleaned_str)
        except ValueError:
            return np.nan
    return np.nan

 

In [13]:
# Lấy danh sách 3 thư mục snapshot
snapshot_folders = [f for f in glob.glob(os.path.join(RAW_DIR, '*')) if os.path.isdir(f)]

# Dùng để lưu kết quả QA của cả 3 snapshot
qa_summary_list = []

print(f"--- BẮT ĐẦU MỤC 3: LÀM SẠCH & CHUẨN HOÁ ---")
print(f"Tìm thấy {len(snapshot_folders)} snapshots để xử lý...")

for folder_path in snapshot_folders:
    snapshot_name = os.path.basename(folder_path)
    print(f"\n--- Đang xử lý: {snapshot_name} ---")

    # === 1. NẠP 4 TỆP THÔ ===
    try:
        listings_df = pd.read_csv(os.path.join(folder_path, 'listings.csv.gz'))
        calendar_df = pd.read_csv(os.path.join(folder_path, 'calendar.csv.gz'))
        reviews_df = pd.read_csv(os.path.join(folder_path, 'reviews.csv.gz'))
        neigh_df = pd.read_csv(os.path.join(folder_path, 'neighbourhoods.csv'))
    except FileNotFoundError as e:
        print(f"LỖI: Thiếu tệp trong {snapshot_name}. Bỏ qua. Lỗi: {e}")
        continue

    # === 2. ÁP DỤNG QUY TẮC LÀM SẠCH & QA ===
    
    # --- Quy tắc 1: Chuẩn hoá GIÁ (dùng hàm 'clean_price') ---
    # (Áp dụng cho 2 tệp có cột 'price')
    listings_df['price_numeric'] = listings_df['price'].apply(clean_price)
    calendar_df['price_numeric'] = calendar_df['price'].apply(clean_price)

    # --- Quy tắc 2: Gắn cờ Outliers (Giá = 0) ---
    listings_df['qa_flag_price_zero'] = (listings_df['price_numeric'] == 0)
    
    # Ghi lại kết quả QA này
    num_zero_price = listings_df['qa_flag_price_zero'].sum()
    qa_summary_list.append({
        'snapshot_date': snapshot_name,
        'rule_id': 'QA001_price_zero',
        'records_affected': num_zero_price,
        'handling_decision': 'Gắn cờ & Loại trừ khỏi tính toán KPI giá'
    })
    
    # --- Quy tắc 3: Chuẩn hoá DATETIME ---
    # (Áp dụng cho 3 tệp có cột ngày tháng)
    listings_df['host_since'] = pd.to_datetime(listings_df['host_since'])
    calendar_df['date'] = pd.to_datetime(calendar_df['date'])
    reviews_df['date'] = pd.to_datetime(reviews_df['date'])
    # --- Quy tắc 4: Xử lý Cột thiếu 100% ---
    # (Xoá 'adjusted_price' khỏi calendar)
    if 'adjusted_price' in calendar_df.columns:
        calendar_df = calendar_df.drop(columns=['adjusted_price'])
        
    # (Xoá 'neighbourhood_group' khỏi neighbourhoods)
    if 'neighbourhood_group' in neigh_df.columns:
        neigh_df = neigh_df.drop(columns=['neighbourhood_group'])
    # --- Quy tắc 5: Kiểm tra tọa độ ngoài Berlin ---
    BERLIN_LAT_MIN, BERLIN_LAT_MAX = 52.30, 52.70
    BERLIN_LON_MIN, BERLIN_LON_MAX = 13.00, 13.80
    listings_df['qa_flag_out_of_berlin'] = (
        (listings_df['latitude'] < BERLIN_LAT_MIN) | 
        (listings_df['latitude'] > BERLIN_LAT_MAX) | 
        (listings_df['longitude'] < BERLIN_LON_MIN) | 
        (listings_df['longitude'] > BERLIN_LON_MAX)
    )
    
    num_out_bounds = listings_df['qa_flag_out_of_berlin'].sum()
    qa_summary_list.append({
        'snapshot_date': snapshot_name,
        'rule_id': 'QA002_coords_out_of_bounds',
        'records_affected': num_out_bounds,
        'handling_decision': 'Gắn cờ & Có thể loại trừ khi vẽ bản đồ'
    })

    # --- Quy tắc 6: Kiểm tra trùng ID ---
    # Đếm các dòng bị trùng ID
    num_duplicates = listings_df.duplicated(subset=['id']).sum()
    
    # Nếu có trùng, ta sẽ drop (giữ lại dòng đầu tiên)
    if num_duplicates > 0:
        listings_df = listings_df.drop_duplicates(subset=['id'], keep='first')
        
    qa_summary_list.append({
        'snapshot_date': snapshot_name,
        'rule_id': 'QA003_duplicate_ids',
        'records_affected': num_duplicates,
        'handling_decision': 'Xóa bỏ các dòng trùng lặp thừa'
    })
    print(f"   > Hoàn tất làm sạch cho {snapshot_name}.")
    print(f"   > (QA001) Phát hiện {num_zero_price} listings giá 0.")
    print(f"   > (QA002) Phát hiện {num_duplicates} dòng trùng lặp ID (đã xoá).")
    print(f"   > (QA003) Phát hiện {num_out_bounds} listings tọa độ sai (ngoài Berlin).") 
    # === 3. LƯU 4 TỆP ĐÃ XỬ LÝ (vào 'processed/') ===
    
    # Tạo thư mục con (ví dụ: 'processed/berlin/15 March, 2025/')
    output_snapshot_dir = os.path.join(PROCESSED_DIR, snapshot_name)
    os.makedirs(output_snapshot_dir, exist_ok=True)

    listings_df.to_csv(os.path.join(output_snapshot_dir, 'listings_processed.csv'), index=False)
    calendar_df.to_csv(os.path.join(output_snapshot_dir, 'calendar_processed.csv'), index=False)
    reviews_df.to_csv(os.path.join(output_snapshot_dir, 'reviews_processed.csv'), index=False)
    neigh_df.to_csv(os.path.join(output_snapshot_dir, 'neighbourhoods_processed.csv'), index=False)
    
    print(f"   > Đã lưu 4 tệp đã xử lý vào: {output_snapshot_dir}")

print("\n--- HOÀN TẤT XỬ LÝ TẤT CẢ SNAPSHOTS ---")

--- BẮT ĐẦU MỤC 3: LÀM SẠCH & CHUẨN HOÁ ---
Tìm thấy 4 snapshots để xử lý...

--- Đang xử lý: 15 March, 2025 ---
   > Hoàn tất làm sạch cho 15 March, 2025.
   > (QA001) Phát hiện 0 listings giá 0.
   > (QA002) Phát hiện 0 dòng trùng lặp ID (đã xoá).
   > (QA003) Phát hiện 0 listings tọa độ sai (ngoài Berlin).
   > Đã lưu 4 tệp đã xử lý vào: ..\processed\berlin\15 March, 2025

--- Đang xử lý: 20 June, 2025 ---
   > Hoàn tất làm sạch cho 20 June, 2025.
   > (QA001) Phát hiện 0 listings giá 0.
   > (QA002) Phát hiện 0 dòng trùng lặp ID (đã xoá).
   > (QA003) Phát hiện 0 listings tọa độ sai (ngoài Berlin).
   > Đã lưu 4 tệp đã xử lý vào: ..\processed\berlin\20 June, 2025

--- Đang xử lý: 21 December, 2024 ---
   > Hoàn tất làm sạch cho 21 December, 2024.
   > (QA001) Phát hiện 0 listings giá 0.
   > (QA002) Phát hiện 0 dòng trùng lặp ID (đã xoá).
   > (QA003) Phát hiện 0 listings tọa độ sai (ngoài Berlin).
   > Đã lưu 4 tệp đã xử lý vào: ..\processed\berlin\21 December, 2024

--- Đang xử l

In [14]:
# --- LƯU BÁO CÁO QA (vào 'reports/') ---

# 1. Chuyển danh sách (list) các kết quả QA thành DataFrame
qa_summary_df = pd.DataFrame(qa_summary_list)

# 2. Định nghĩa đường dẫn lưu báo cáo
qa_report_path = os.path.join(REPORTS_DIR, 'qa_summary_berlin.csv')

# 3. Lưu file
qa_summary_df.to_csv(qa_report_path, index=False, encoding='utf-8-sig')
print(f"Đã lưu báo cáo QA tại: {qa_report_path}")

Đã lưu báo cáo QA tại: ..\reports\qa_summary_berlin.csv
